In [1]:
import pandas as pd

# 1. CARGA DE DATOS
print("Cargando archivos...")
users = pd.read_parquet('users_info.parquet')
h_sessions = pd.read_parquet('horses_sessions_info.parquet')
p_sessions = pd.read_parquet('prods_sessions_info.parquet')

# Cargamos los catálogos de caballos
h_list_1 = pd.read_parquet('equinenow_horses_listings_limpio.parquet')
h_list_2 = pd.read_parquet('horsedeals_horses_listings_limpio.parquet')

# --- UNIFICACIÓN DE COLUMNAS (Mayúsculas a Minúsculas) ---
# Pasamos 'Horse_ID' a 'horse_id' y 'Price' a 'price' para que coincida con las sesiones
h_list_1 = h_list_1.rename(columns={'Horse_ID': 'horse_id', 'Price': 'price'})
h_list_2 = h_list_2.rename(columns={'Horse_ID': 'horse_id', 'Price': 'price'})

# Unimos los catálogos y quitamos duplicados
h_listings = pd.concat([h_list_1, h_list_2]).drop_duplicates(subset='horse_id')

# Cargamos productos (ponemos precio en minúscula también por si acaso)
p_listings = pd.read_parquet('doversaddlery_products_listing_limpio.parquet')
if 'Price' in p_listings.columns:
    p_listings = p_listings.rename(columns={'Price': 'price'})

print("✅ ¡Archivos cargados y nombres normalizados!")

# 2. NORMALIZACIÓN DE USUARIOS (Desarmar la mochila job_info)
df_jobs = pd.json_normalize(users['job_info'])
users_clean = pd.concat([users.drop(columns=['job_info']), df_jobs], axis=1)

# 3. LÓGICA DE TARGET (Reglas de negocio)

# A. Identificar quiénes compraron (Purchase)
purchase_ids = pd.concat([
    h_sessions[h_sessions['event_type'] == 'purchase']['user_id'],
    p_sessions[p_sessions['event_type'] == 'purchase']['user_id']
]).unique()

# B. Identificar quiénes agregaron al carrito (Add to cart)
cart_ids = pd.concat([
    h_sessions[h_sessions['event_type'] == 'add_to_cart']['user_id'],
    p_sessions[p_sessions['event_type'] == 'add_to_cart']['user_id']
]).unique()

# C. Identificar quiénes son ORO (Compra de caballo > $50,000)
# Ahora 'horse_id' y 'price' están en minúsculas en todos lados
h_purchases = h_sessions[h_sessions['event_type'] == 'purchase'].merge(h_listings[['horse_id', 'price']], on='horse_id')
oro_ids = h_purchases[h_purchases['price'] > 50000]['user_id'].unique()

# 4. CLASIFICACIÓN FINAL
def clasificar_lead(uid):
    if uid in oro_ids: return 'Oro'
    if uid in purchase_ids: return 'Plata'
    if uid in cart_ids: return 'Bronce'
    return 'Interesado'

users_clean['target_lead'] = users_clean['user_id'].apply(clasificar_lead)

print("\n--- RESULTADO FINAL DE LA SEMANA 1 ---")
print(users_clean['target_lead'].value_counts())

Cargando archivos...
✅ ¡Archivos cargados y nombres normalizados!

--- RESULTADO FINAL DE LA SEMANA 1 ---
target_lead
Interesado    188037
Plata          11811
Oro              152
Name: count, dtype: int64


In [ ]:
# Guardamos el resultado final para entregarlo al equipo
#users_clean.to_parquet('users_with_target_week1.parquet')
#print("✅ Archivo final 'users_with_target_week1.parquet' generado.")

✅ Archivo final 'users_with_target_week1.parquet' generado.
